## Demo

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

from decouple import config
from fastembed import (
    TextEmbedding, 
    SparseTextEmbedding, 
    LateInteractionTextEmbedding
)
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance,
    Fusion,
    VectorParams, 
    SparseVectorParams,
    SparseIndexParams,
    MultiVectorConfig,
    MultiVectorComparator,
    Modifier,
    HnswConfigDiff
)

from rag.models import (
    DenseModelConfig, 
    SparseModelConfig,
    RerankingModelConfig,
    DenseSearchManager,
    SparseSearchManager,
    HybridFusionSearchManager, 
    HybridRerankingSearchManager, 
    Metadata
)


QDRANT_URL = config('QDRANT_URL')
QDRANT_API_KEY = config('QDRANT_API_KEY')
NVIDIA_API_KEY = config('NVIDIA_API_KEY')

### Client

In [2]:
qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

print(qdrant_client.get_collections())

collections=[]


### Models

In [3]:
dense_model_name = 'BAAI/bge-small-en-v1.5'
dense_model = TextEmbedding(model_name=dense_model_name)

sparse_model_name = 'Qdrant/bm25'
sparse_model = SparseTextEmbedding(
    model_name=sparse_model_name,
    k1=1.5,
    b=0.75
)

reranking_model_name = 'colbert-ir/colbertv2.0'
reranking_model = LateInteractionTextEmbedding(reranking_model_name)

dense_model_config = DenseModelConfig(
    name=dense_model_name,
    vector_params=VectorParams(
        size=384,
        distance=Distance.COSINE,
        hnsw_config=HnswConfigDiff(
            m=16,
            ef_construct=128,
            on_disk=True
        ),
        on_disk=True
    )
)

sparse_model_config = SparseModelConfig(
    name=sparse_model_name,
    sparse_vector_params=SparseVectorParams(
        index=SparseIndexParams(
            on_disk=True
        ),
        modifier=Modifier.IDF
    )
)

reranking_model_config = RerankingModelConfig(
    name=reranking_model_name,
    vector_params=VectorParams(
        size=128,
        distance=Distance.COSINE,
        hnsw_config=HnswConfigDiff(
            m=0     # disable HNSW
        ),
        on_disk=True,
        multivector_config=MultiVectorConfig(
            comparator=MultiVectorComparator.MAX_SIM,
        )
    )
)

### Documents

In [4]:
documents = [
    'FastEmbed is lighter than Transformers & Sentence-Transformers.',
    'FastEmbed is supported by and maintained by Qdrant.',
]
metadatas = [
    Metadata(
        id=i + 1,
        text=document
    )
    for i, document in enumerate(documents)
]

dense_embeddings = list(dense_model.embed(documents))
sparse_embeddings = list(sparse_model.embed(documents))
reranking_embeddings = list(reranking_model.embed(documents))

### Indexing

#### Dense

In [5]:
dense_search = DenseSearchManager(
    qdrant_client=qdrant_client, 
    dense_model_config=dense_model_config
)
dense_collection_name = 'dense_collection'

dense_search.create_collection(dense_collection_name)
dense_search.upsert(
    dense_collection_name,
    dense_embeddings,
    metadatas
)

'completed'

#### Sparse

In [6]:
sparse_search = SparseSearchManager(
    qdrant_client=qdrant_client, 
    sparse_model_config=sparse_model_config
)
sparse_collection_name = 'sparse_collection'

sparse_search.create_collection(sparse_collection_name)
sparse_search.upsert(
    sparse_collection_name,
    sparse_embeddings,
    metadatas
)

'completed'

#### Hybrid - Fusion

In [ ]:
fusion_search = HybridFusionSearchManager(
    qdrant_client=qdrant_client, 
    dense_model_config=dense_model_config,
    sparse_model_config=sparse_model_config
)
fusion_collection_name = 'fusion_collection'

fusion_search.create_collection(fusion_collection_name)
fusion_search.upsert(
    fusion_collection_name, 
    dense_embeddings, 
    sparse_embeddings,
    metadatas
)

'completed'

#### Hybrid - Reranking

In [ ]:
reranking_search = HybridRerankingSearchManager(
    qdrant_client=qdrant_client, 
    dense_model_config=dense_model_config,
    sparse_model_config=sparse_model_config,
    reranking_model_config=reranking_model_config
)
reranking_collection_name = 'reranking_collection'

reranking_search.create_collection(reranking_collection_name)
reranking_search.upsert(
    reranking_collection_name, 
    dense_embeddings, 
    sparse_embeddings,
    reranking_embeddings,
    metadatas
)

'completed'

### Query

In [11]:
query_document = 'Who is lighter than Transformers?'

query_dense_embedding = next(iter(dense_model.embed(query_document)))
query_sparse_embedding = next(iter(sparse_model.embed(query_document)))
query_reranking_embedding = next(iter(reranking_model.embed(query_document)))

### Search

#### Dense

In [12]:
dense_scored_points = dense_search.search(
    dense_collection_name,
    query_dense_embedding,
    1
)

print(*dense_scored_points, sep='\n')

id='c3181a05-9953-4b81-a491-15b9f1b3208d' version=0 score=0.7509401 payload={'id': 1, 'text': 'FastEmbed is lighter than Transformers & Sentence-Transformers.'} vector=None shard_key=None order_value=None


#### Sparse

In [13]:
sparse_scored_points = sparse_search.search(
    sparse_collection_name,
    query_sparse_embedding,
    1
)

print(*sparse_scored_points, sep='\n')

id='2bd53b67-23b4-4a84-8f47-12782ef07e2a' version=0 score=4.1631217 payload={'id': 1, 'text': 'FastEmbed is lighter than Transformers & Sentence-Transformers.'} vector=None shard_key=None order_value=None


#### Hybrid - Fusion

In [14]:
fusion_scored_points = fusion_search.search(
    fusion_collection_name,
    query_dense_embedding,
    query_sparse_embedding,
    Fusion.RRF,
    1
)

print(*fusion_scored_points, sep='\n')

id='2c0b2e24-1435-451c-8dc0-298e1c710e53' version=0 score=1.0 payload={'id': 1, 'text': 'FastEmbed is lighter than Transformers & Sentence-Transformers.'} vector=None shard_key=None order_value=None


#### Hybrid - Reranking

In [15]:
reranking_scored_points = reranking_search.search(
    reranking_collection_name,
    query_dense_embedding,
    query_sparse_embedding,
    query_reranking_embedding,
    2,
    1
)

print(*reranking_scored_points, sep='\n')

id='2d78e6a2-e0a4-4380-afe8-7d054612d469' version=0 score=5.8587246 payload={'id': 1, 'text': 'FastEmbed is lighter than Transformers & Sentence-Transformers.'} vector=None shard_key=None order_value=None


In [ ]:
# NOTE: use with SPLADE

import json
from fastembed.sparse import SparseEmbedding
from tokenizers import Tokenizer


def get_tokens_to_weights(sparse_embedding: SparseEmbedding, tokenizer: Tokenizer) -> dict:
    token_weight_dict = {}
    
    for i in range(len(sparse_embedding.indices)):
        token = tokenizer.decode([sparse_embedding.indices[i]])
        weight = sparse_embedding.values[i]
        token_weight_dict[token] = weight

    # sort by weights
    token_weight_dict = dict(sorted(token_weight_dict.items(), key=lambda item: item[1], reverse=True))
    
    return token_weight_dict


index = 0
tokenizer = Tokenizer.from_pretrained('Qdrant/SPLADE_PP_en_v1')
tokens_to_weights = get_tokens_to_weights(sparse_embeddings[index], tokenizer)

print(json.dumps(tokens_to_weights, indent=4))

### LLM

In [21]:
client = OpenAI(
  base_url = 'https://integrate.api.nvidia.com/v1',
  api_key = NVIDIA_API_KEY
)

content = (
    'You are a helpful assistant that answers given question using ONLY PROVIDED CONTEXT.\n'
    'You are not allowed to use any previous knowledge.\n\n'

    'The output should be a well-formatted JSON object that conforms to the example below\n'
    '("answer" is either string or null):\n'
    '{"answer": "some answer"}\n\n'

    'If you don\'t know the answer, return:\n'
    '{"answer": null}\n\n'

    '<context_start>\n'
    f'{reranking_scored_points[0].payload['text']}\n'
    '<context_end>\n\n'

    '<question_start>\n'
    f'{query_document}\n'
    '<question_end>'
)

completion = client.chat.completions.create(
    model='meta/llama-3.1-405b-instruct',
    messages=[{
        'role': 'user', 
        'content': content
    }],
    temperature=0.2,
    max_tokens=1024,
    stream=False
)
print(completion)

ChatCompletion(id='chat-5f1ee1ebe77d4e9ca3fb01051219d96a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"answer": "FastEmbed"}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), stop_reason=None)], created=1736019507, model='meta/llama-3.1-405b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=7, prompt_tokens=115, total_tokens=122, completion_tokens_details=None, prompt_tokens_details=None))
